<img src='img/EUMETSAT_Logo_WhiteonBlue.png' alt='logo EUMETSAT' align='right' width='30%'></img>

## <span style="color:blue">**MTG LI data explore**</span>

<hr>

## Explore & Display Meteosat Third Generation (MTG) LI data
### Level 2 Flash Product

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
This notebook has the following requirement:
    
the notebook describes operations on an MTG LI product that can be downloaded as described in the notebook [**Link to notebook on GitHub**](https://gitlab.eumetsat.int/eumetlab/data-services/eumdac_data_store/-/blob/master/1_6_MTG_LI_data_access.ipynb?ref_type=heads)
    

</div>
<hr>

### About

This notebook provides you with an introduction to Level 2 data from the Lightning Imager instrument. The MTG LI was launched in December 2022 and is on board of the MTG-I1 satellite and provides continuous optical observations of lightning. <br>

Basic Facts about LI Instrument:
> **Spatial resolution at Nadir**: `4.5 km (variable throughout the Field of View, FOV)`<br>
> **Sectral Band**: `777.4 nm and 1.9 nm narrow band` <br>
> **Frame rate**: `1 ms` <br>



Five products are disseminated as level 2 data:

1) Point data:
   * Group - 
This is an optical pulse detected by LI in one frame (that should be considered as a counterpart of the ground-detected stroke)
   * Flash - The collection of groups that are correlated in space and time  (that should be considered as counterparts of the ground-detected flashes
>
3) Gridded data:
   * Accumulated Flash (keeps track of the density of events within sequences)
   * Accumulated Flash Area (Allows to keep track of the areas touched by flashes)
   * Accumulated Flash Radiance (representing the total radiance detected within a certain pixel from multiple events)
  
Reference: <a href="https://user.eumetsat.int/resources/user-guides/mtg-li-level-2-data-guide#ID-Group-and-flash-data">link</a>.
 

### Outline
This notebook is focusing on the **Level 2 Flash product**. It gives guidance on:

1) how to load data and inspect its contents <br>
2) how to display a map with the spatial distribution of lightning flashes across parts of the globe for a given date, with colors indicating the time of day when the flashes occur (temporal distribution of the flashes) 

### Data used

| Product Description  | Data Store collection ID| Product Navigator | 
|:--------------------:|:-----------------------:|:-------------:|
| MTG LI level-2 Flash | EO:EUM:DAT:0691         | <a href="https://data.eumetsat.int/data/map/EO:EUM:DAT:0691">link</a> |


A product is made of the following files:
* **Body chunks** (NetCDF): One body chunk containing the measured data values in NetCDF structure.
* Trailer chunk (NetCDF): One trailer chunk that contains at least the list of preceding body chunks that have been produced for the dataset. 
* Quicklooks (PNG): Images related to data that enable fast viewing for the purpose of selection by archive users.
* manifest.xml: The manifest contains further information about the SIP, especially the relationships among the different items constituting the package.
* EOPMetadata.xml: The EOPMetadata file contains metadata about the product based on the OGC Earth Observation Metadata Profile of Observations and Measurements (OGC document 10-157r4).


### Module Content:
- [1 - Load and Inspect MTG LI data (FLASHES)](#LI_load)
- [2 - Data display](#Data_display)
- [2.1 - Select time range](#Select_time)
- [2.1 - Select Region Of Interest](#Select_region)


<hr>

##### Load required libraries

 We begin by importing all of the libraries that we need to run this notebook. If you have built your python using the environment file provided in this repository, then you should have everything you need. For more information on building environment, please see the repository

In [ ]:
from glob import glob
from satpy.scene import Scene
from AuxiliaryFunctions import plot_lmap_from_filenames
from AuxiliaryFunctions import filter_filenames
from datetime import datetime

## <a id='LI_load'></a>1. Load and Inspect LI data

From the [EUMETSAT Data Store](https://data.eumetsat.int/data/map/EO:EUM:DAT:MSG:HRSEVIRI), we downloaded LI data sensed for a specific day. The **Data_folder** variable should point to the folder containing LI LFL products, unzipped by the LI data download Notebook. As the data is contained in the **BODY** files, and since the data files have 'BODY' in their filenames, we will use the wildcard *BODY* to search for the relevant files within the folder. Another wildcard `/*/` is needed as each product is unzipped into its own subdirectory.

In [ ]:
Data_folder='downloads/*BODY*.nc'

We save the list with the filenames in the folder into a variable called filenames, and sort the list:

In [ ]:
filenames = glob(Data_folder)
filenames.sort()

To access a file for a specific date, you can specify its position in the list with a specific index, e.g. `[2]` (indexing starts from 0).

In [ ]:
first_file = [filenames[0]]
first_file

The Python package [satpy](https://satpy.readthedocs.io/en/stable/) supports reading and loading data from many input files, like from LI data. To check the data readers available from satpy, you can uncomment the following line. For LI data, we are interested in `li_l2_nc` reader.

In [ ]:
#import satpy
#satpy.available_readers()

Next, you can use the `Scene` constructor from the [satpy](https://satpy.readthedocs.io/en/stable/index.html) library to load the data files. Once loaded, a `Scene` object represents a single geographic region of data, typically at a single continuous time range.

You have to specify the two keyword arguments `reader` and `filenames` in order to successfully load a scene. 

In [ ]:
scn = Scene(reader="li_l2_nc", 
            filenames=first_file)

A `Scene` object loads all band information of a satellite image. With the function `available_dataset_names()`, you can see the available datasets available in the LI data file.

In [ ]:
scn.available_dataset_names()

In this JN we demonstrate how to obtain spatial distribution of lightning flashes across parts of the globe with colors indicating the time of day when the flashes occur. Therefore, we will be loading the 'flash_time' dataset. This variable corresponds to UTC time of the first group in the flash. Additionaly, to map the location of each flash, we also need to load the corresponding  latitude and longitude variables.

In [ ]:
latitude_dataset_name='latitude'
longitude_dataset_name='longitude'
time_dataset_name='flash_time'
scn.load([latitude_dataset_name, longitude_dataset_name, time_dataset_name])
latitude_dataset_uncorr = scn[latitude_dataset_name].compute()
longitude_dataset_uncorr = scn[longitude_dataset_name].compute()
time_dataset = scn[time_dataset_name].compute()

We can also check the start and end time of the data within the file:

In [ ]:
print(time_dataset.start_time)
print(time_dataset.end_time)

When using data retrieved from the EUMETSAT Data Store these variables should indicate a time range (difference between start_time and end_time of 10 minutes, since the stored data files correspond to 10-minute-long files (containing 60 windows of 10 sec each)). On the opposite, disseminated files correspond to 10 sec of measurements.

## <a id='Data_display'></a>2. Data display

### <a id='Select_time'></a>2.1 Select Time Range

This notebook is prepared to display LI data covering 1 day. Yet, you can narrow the time range of the data displayed.

Current Time Range:

* start_dt = 2024-09-01 00:00
* end_dt = 2024-09-02 00:00

How to Modify:
* Set a new starting time by providing new numeric values in the format: start_dt = datetime (year, month, day, hour, minute).
* Set a new ending time by providing new numeric values in the format: end_dt = datetime (year, month, day, hour, minute). <br>
   For example, to set the range of times to be displayed between start time September 1, 2024, at 16:30 and 18:30, you would write:<br>
   start_dt = datetime (2024, 9, 1, 16, 30).<br>
   end_dt = datetime (2024, 9, 1, 18, 30).

  Note: please be aware that in **minute you should choose 0 / 10 /20 /30 /40 /50**

<div class="alert alert-block alert-success">
<b>You may want to modify the next cell:</b><br />
</div>

In [ ]:
start_dt = datetime(2024, 10, 29, 0, 0)
end_dt = datetime(2024, 10, 30, 0, 0)

In [ ]:
filtered_files = filter_filenames(filenames, start_dt, end_dt)

In [ ]:
area_nswe = None
#area_nswe = [80, 20, -30, 50]

Optionally we can also plot the boundaries of the field-of-views of the 4 LI cameras by setting the `plot_fov` below `True`. Please consider that especially if the data is not corrected for the parallax effect, a few LI lightning locations can also appear slightly outside the outer boundaries of the field-of-view polygons. 

In [ ]:
plot_fov = True

In the next cell we are setting the title and calling the function that produces the display of the flash data with colors indicating the time of day when the flashes occurr, as depicted by the color bar on the right. This functions returns to use figure and axes associated with the plot.

In [ ]:
title = str(start_dt.date())+ ' ' + str(time_dataset_name)
fig,ax = plot_lmap_from_filenames(filtered_files, time_dataset_name, area_nswe, title, plot_fov=plot_fov)

### <a id='Select_region'></a>2.2 Select Region Of Interest

The notebook is set by default to display lightning data over the entire area covered by the Lightning Sensor.
However, if you would like to focus on a specific Region Of Interest (ROI), you can do so by entering the latitude and longitude coordinates in the variable provided. So:
* If you prefer to use the default area, leave the 'line area_nswe = None' uncommented.
* If you want to select your own ROI, uncomment the line for the ROI and enter your specific coordinates. The format should be **[lat_max, lat_min, lon_min, lon_max]**

<div class="alert alert-block alert-success">
<b>You may want to  modify the next cell:</b><br />
</div>

In [ ]:
#area_nswe = None
#area_nswe = [45, 35, 5, 20] # Italy
area_nswe = [45, 30, -10, 10] # Spain

In the next cell, we set the title and call the function that produces the display of the flash data with colours indicating the time of day when the flashes occur, as depicted by the colour bar on the right. This function returns to use figures and axes associated with the plot.

<div class="alert alert-block alert-success">
<b>You may want to adapt the title of the plot in the next cell:</b><br />
</div>

In [ ]:
title = 'Flash Map for Iberian Peninsula from MTG LI ' + '[' + start_dt.strftime("%Y-%m-%d") + ' (' + start_dt.strftime("%H:%M") + '-' + start_dt.strftime("%H:%M")+ ') '+ ']'

fig,ax=plot_lmap_from_filenames(filtered_files, time_dataset_name, area_nswe, title)

You may want to save the image to a file (PNG format). You can do that by running the below cell. This will save the file into the current folder, but if you want it saved elsewhere, you can change the output_filename accordingly. 

In [ ]:
output_filename = title + ".png"
fig.savefig(output_filename)

<div class="alert alert-info" role="alert">

Please try to analyze the obtained map, keeping the following questions in mind as part of your interpretation: 

Which regions on the map show the highest concentration of lightning flashes?

What time of day do most lightning flashes occur in West and Central Africa, according to the color-coded map? How does this compare to the timing of the lightning maximum in South America?

Based on the map, what relationship can you infer between the time of day and the occurrence of lightning ?

What could explain the relatively lower concentration of lightning flashes at higher latitudes compared to the equatorial regions in this dataset?

What can you say about the lightning distribution between the land and the oceans? What could explain the difference?


<br>

<br>

<hr>


    
<p style="text-align:right;"> <a href='https://training.eumetsat.int'>EUMETSAT Training</a> | <a href='mailto:training@eumetsat.int'>Contact the training team</a></p>